<a href="https://colab.research.google.com/github/shunrei9841-sudo/Guadalupe/blob/main/Copia_de_Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pprint
from fractions import Fraction

# --- 1. PARÁMETROS DEL PROBLEMA ---
L = 2  # Longitud del dominio (0 a 2 en x y y)
h = 2/3  # Tamaño de paso (h = 2/3)

# Calcular el número de pasos internos
# N = L/h
N = int(L / h) # N = 2 / (2/3) = 3.
# Nodos internos: i=1, ..., N-1 y j=1, ..., N-1. (i=1, 2 y j=1, 2)
# Tenemos 4 incógnitas: u_11, u_21, u_12, u_22

print(f"Dominio: [0, {L}] x [0, {L}]")
print(f"Tamaño de paso (h): {Fraction(h)}")
print(f"Número total de divisiones (N): {N}\n")

# --- 2. DEFINICIÓN DE COORDENADAS DE LOS NODOS ---
# Coordenadas: x_i = i*h, y_j = j*h
x_coords = [i * h for i in range(N + 1)]
y_coords = [j * h for j in range(N + 1)]

# Imprimir coordenadas
print(f"Coordenadas X: {[Fraction(val) for val in x_coords]}")
print(f"Coordenadas Y: {[Fraction(val) for val in y_coords]}\n")

# --- 3. CÁLCULO DE VALORES DE FRONTERA (Fijos) ---

# Inicializar la matriz U (tamaño (N+1) x (N+1))
# U[i, j] representa u(x_i, y_j)
U = np.zeros((N + 1, N + 1), dtype=float)

# 3.1. Frontera Izquierda (x = 0): u(0, y) = 0
# U[0, j] = 0 (ya inicializado)

# 3.2. Frontera Inferior (y = 0): u(x, 0) = 0
# U[i, 0] = 0 (ya inicializado)

# 3.3. Frontera Derecha (x = 2): u(2, y) = y(2 - y)
for j in range(1, N): # j = 1, 2
    y = y_coords[j]
    U[N, j] = y * (2 - y) # U[3, j]

# 3.4. Frontera Superior (y = 2): u(x, 2) = f(x)
def f_top(x):
    if 0 < x < 1:
        return x
    elif 1 <= x < 2:
        return 2 - x
    else:
        return 0 # Esquinas

for i in range(1, N): # i = 1, 2
    x = x_coords[i]
    U[i, N] = f_top(x) # U[i, 3]

# Mostrar la matriz de frontera (incluye esquinas, que son 0)
print("Matriz de Valores de Frontera (U[i, j]):")
print("---------------------------------------")
# Convertir a Fraction para mayor precisión al mostrar
U_frac = np.array([[Fraction(val).limit_denominator(100) for val in row] for row in U])
print(U_frac)
print("\nValores Frontera Clave (como fracciones):")
print(f"u_31: {U_frac[3, 1]}, u_32: {U_frac[3, 2]} (8/9)")
print(f"u_13: {U_frac[1, 3]} (2/3), u_23: {U_frac[2, 3]} (2/3)\n")

# --- 4. IMPLEMENTACIÓN DEL MÉTODO DE JACOBI ---

# El método de Jacobi para la Ecuación de Laplace (Laplaciano de 5 puntos) es:
# u_ij^(k+1) = 1/4 * (u_(i+1)j^k + u_(i-1)j^k + u_i(j+1)^k + u_i(j-1)^k)

# Inicialización k=0: Usamos 0 para los nodos interiores
# U es nuestra aproximación actual u^k. U_new es u^(k+1).
U_old = U.copy()

# Aislamos solo los nodos internos (i=1, 2; j=1, 2)
def get_internal_values(U_matrix):
    return U_matrix[1:N, 1:N].flatten()

# Iteración k=0 (Aproximación inicial)
# U_old ya tiene 0 en los internos.
u_0 = get_internal_values(U_old)
print(f"Aproximación inicial (u^(0)): {[Fraction(v).limit_denominator(100) for v in u_0]}\n")

# --- PRIMERA ITERACIÓN (k=1) ---
U_new = U_old.copy() # Copia de U_old (incluye fronteras y ceros internos)
k = 1

for i in range(1, N): # i = 1, 2
    for j in range(1, N): # j = 1, 2
        # Aplicar la fórmula de Jacobi
        U_new[i, j] = 0.25 * (U_old[i+1, j] + U_old[i-1, j] + U_old[i, j+1] + U_old[i, j-1])

u_1 = get_internal_values(U_new)
print(f"--- Iteración {k} (u^({k})) ---")
print(f"u_11 = 1/4 * (0 + 0 + 1/6 + 2/9) = {Fraction(u_1[0]).limit_denominator(100)} (~{u_1[0]:.4f})")
print(f"u_21 = 1/4 * (0 + 0 + 7/18 + 8/9) = {Fraction(u_1[1]).limit_denominator(100)} (~{u_1[1]:.4f})")
print(f"u_12 = 1/4 * (0 + 0 + 2/3 + 0) = {Fraction(u_1[2]).limit_denominator(100)} (~{u_1[2]:.4f})")
print(f"u_22 = 1/4 * (0 + 0 + 2/3 + 8/9) = {Fraction(u_1[3]).limit_denominator(100)} (~{u_1[3]:.4f})")
print(f"\nResultado u^({k}) (u_11, u_21, u_12, u_22): {[Fraction(v).limit_denominator(100) for v in u_1]}")


# --- SEGUNDA ITERACIÓN (k=2) ---
U_old = U_new.copy() # La nueva aproximación se convierte en la anterior
k = 2

# Recalcular U_new usando los valores U_old (u^1)
for i in range(1, N): # i = 1, 2
    for j in range(1, N): # j = 1, 2
        U_new[i, j] = 0.25 * (U_old[i+1, j] + U_old[i-1, j] + U_old[i, j+1] + U_old[i, j-1])

u_2 = get_internal_values(U_new)
print(f"\n--- Iteración {k} (u^({k})) ---")
# Imprimir valores clave (usando U_old para mostrar de dónde vienen los términos)
print(f"u_11 = 1/4 * (u_21^1 + u_01 + u_12^1 + u_10) = 1/4 * ({Fraction(U_old[2, 1])} + 0 + {Fraction(U_old[1, 2])} + 0) = {Fraction(u_2[0]).limit_denominator(100)} (~{u_2[0]:.4f})")
print(f"u_21 = 1/4 * (u_31 + u_11^1 + u_22^1 + u_20) = 1/4 * ({Fraction(U_old[3, 1])} + {Fraction(U_old[1, 1])} + {Fraction(U_old[2, 2])} + 0) = {Fraction(u_2[1]).limit_denominator(100)} (~{u_2[1]:.4f})")
print(f"u_12 = 1/4 * (u_22^1 + u_02 + u_13 + u_11^1) = 1/4 * ({Fraction(U_old[2, 2])} + 0 + {Fraction(U_old[1, 3])} + {Fraction(U_old[1, 1])}) = {Fraction(u_2[2]).limit_denominator(100)} (~{u_2[2]:.4f})")
print(f"u_22 = 1/4 * (u_32 + u_12^1 + u_23 + u_21^1) = 1/4 * ({Fraction(U_old[3, 2])} + {Fraction(U_old[1, 2])} + {Fraction(U_old[2, 3])} + {Fraction(U_old[2, 1])}) = {Fraction(u_2[3]).limit_denominator(100)} (~{u_2[3]:.4f})")
print(f"\nResultado u^({k}) (u_11, u_21, u_12, u_22): {[Fraction(v).limit_denominator(100) for v in u_2]}")

Dominio: [0, 2] x [0, 2]
Tamaño de paso (h): 6004799503160661/9007199254740992
Número total de divisiones (N): 3

Coordenadas X: [Fraction(0, 1), Fraction(6004799503160661, 9007199254740992), Fraction(6004799503160661, 4503599627370496), Fraction(2, 1)]
Coordenadas Y: [Fraction(0, 1), Fraction(6004799503160661, 9007199254740992), Fraction(6004799503160661, 4503599627370496), Fraction(2, 1)]

Matriz de Valores de Frontera (U[i, j]):
---------------------------------------
[[Fraction(0, 1) Fraction(0, 1) Fraction(0, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(0, 1) Fraction(2, 3)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(0, 1) Fraction(2, 3)]
 [Fraction(0, 1) Fraction(8, 9) Fraction(8, 9) Fraction(0, 1)]]

Valores Frontera Clave (como fracciones):
u_31: 8/9, u_32: 8/9 (8/9)
u_13: 2/3 (2/3), u_23: 2/3 (2/3)

Aproximación inicial (u^(0)): [Fraction(0, 1), Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)]

--- Iteración 1 (u^(1)) ---
u_11 = 1/4 * (0 + 0 + 1/6 + 2/9) = 0 (~0.00